In [1]:
from langchain_community.utilities import SerpAPIWrapper
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Markdown
import textwrap
import os
from langchain_core.tools import Tool

In [2]:
# using v2rayn
os.environ["HTTP_PROXY"] = "http://127.0.0.1:10809"
os.environ["HTTP_PROXYS"] = "http://127.0.0.1:10809"


# AI model
model = "gemini-1.5-flash"

In [3]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
chat = ChatGoogleGenerativeAI(model=model,
                              google_api_key=os.getenv("GOOGLE_API_KEY"))


search = SerpAPIWrapper(serpapi_api_key=os.getenv("SERP_API_KEY"))
search_tool = Tool(
    name="serpapi",
    description="A tool that uses the SerpAPI to search the web.",
    func=search.run,
)

tools = [search_tool]


chat_with_tools = chat.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            "You are a helpful assistant. Answer all questions to the best of your ability. When I want to search the web, I will ask you to use the SerpAPI tool."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat_with_tools


chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [25]:
res = chain_with_message_history.invoke(
    {"input": "搜索英国最近的新闻"},
    {"configurable": {"session_id": "unused"}},
)

res

Parent run fafcc990-c575-4208-a594-a902f17691b8 not found for run 2dd8f9dd-5d86-43c4-8bc5-1742fbe2fd33. Treating as a root run.


AIMessage(content='', additional_kwargs={'function_call': {'name': 'serpapi', 'arguments': '{"q": "latest news in UK"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-00c30e6a-7a2a-46e2-ae30-4338f8a75cab-0', tool_calls=[{'name': 'serpapi', 'args': {'q': 'latest news in UK'}, 'id': '59db8738-cd26-4ea7-b84d-712fc1519784'}], usage_metadata={'input_tokens': 212, 'output_tokens': 17, 'total_tokens': 229})

In [22]:
res = chain_with_message_history.invoke(
    {"input": "我刚才问的什么问题"},
    {"configurable": {"session_id": "unused"}},
)
to_markdown(res.content)

Parent run d33ff0ca-504c-43b4-9850-6706eb59199d not found for run cede7d9a-d804-4118-a9bd-f64d7917fd6f. Treating as a root run.


> 你刚才问了“搜索英国最近的新闻”。 


In [26]:
res.tool_calls

[{'name': 'serpapi',
  'args': {'q': 'latest news in UK'},
  'id': '59db8738-cd26-4ea7-b84d-712fc1519784'}]